In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

,gejala,kerusakan
0,Mesin tidak bisa dinyalakan,Mesin
1,Bau asap keluar dari mesin,Mesin
2,Bunyi mesin aneh,Mesin
3,Mesin bergetar secara tidak normal,Kelistrikan
4,Motor panas berlebihan,Kelistrikan


In [7]:
# Membagi data menjadi data latih dan data uji
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Tokenisasi teks
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['gejala'])

# Mengonversi teks ke dalam urutan angka
train_sequences = tokenizer.texts_to_sequences(train_data['gejala'])
test_sequences = tokenizer.texts_to_sequences(test_data['gejala'])

# Padding urutan sehingga memiliki panjang yang sama
max_len = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in test_sequences))
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

In [14]:
# Membuat model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=16, input_length=max_len),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Mengonversi label menjadi angka biner (0 atau 1)
train_labels = (train_data['kerusakan'] == 'kerusakan_mesin').astype(int)
test_labels = (test_data['kerusakan'] == 'kerusakan_mesin').astype(int)

In [15]:
# Melatih model
model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels))

# Evaluasi model
loss, accuracy = model.evaluate(test_padded, test_labels)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6689 - accuracy: 1.0000 - val_loss: 0.6796 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 91ms/step - loss: 0.6620 - accuracy: 1.0000 - val_loss: 0.6753 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 81ms/step - loss: 0.6549 - accuracy: 1.0000 - val_loss: 0.6710 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 83ms/step - loss: 0.6478 - accuracy: 1.0000 - val_loss: 0.6667 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 74ms/step - loss: 0.6406 - accuracy: 1.0000 - val_loss: 0.6623 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 85ms/step - loss: 0.6332 - accuracy: 1.0000 - val_loss: 0.6579 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 82ms/step - loss: 0.6257 - accuracy: 1.0000 - val_loss: 0.6531 - val_accuracy: 1.0000
Epoch 8/10
1/1 [=============

In [12]:
# Mapping indeks kategori ke nama kategori
kategori_mapping = {0: 'Mesin', 1: 'Kelistrikan', 2: 'Sistem Rem', 3: 'Kerusakan Lainnya'}

In [13]:
# Tampilkan hasil prediksi
df['prediksi'] = [kategori_mapping[pred.argmax()] for pred in predictions]
print(df[['gejala', 'kerusakan', 'prediksi']])

ValueError: Length of values (3) does not match length of index (9)